In [1]:
import torch.nn as nn
import torch
from torchvision import transforms, datasets
import numpy as np
import math

## Image Preprocessing

They only mean center so we found the mean pixel value of faces and normalize with that.

In [2]:
data_transform = transforms.Compose([transforms.Grayscale(),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.5089547997389491],
                                     std=[1])])
allImages = datasets.ImageFolder(root='./training',transform = data_transform)
label_mapping = torch.FloatTensor([float(clazz) for clazz in allImages.classes])
# label_mappin
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# make it output from [0,1] rather than [1900,2010]
label_mapping_scaled = (label_mapping - label_mapping.min())/(label_mapping.max() - label_mapping.min())

In [4]:
dataloader = torch.utils.data.DataLoader(allImages,batch_size = 128, shuffle=True)

In [5]:
class ResNet(nn.Module):
    
    def __init__(self, n_layers, final_output, bottleneck = False):
        super(ResNet,self).__init__()
        self.conv_params = {'kernel_size': 3, 'padding': 1}
        self.width = 186
        self.height = 171
        self.layer_dict = {
            18: [2,2,2,2],
            34: [3,4,6,3]
        }
        self.layers = {}
        
        in_channels = 1
        out_channels = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 7, stride = 2, padding = 3),
            nn.BatchNorm2d(num_features = out_channels),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1, dilation = 1)
        )
        self.width = self.width / 2
        self.width = math.floor( (self.width - 1) / 2 + 1 )
        self.height = self.height / 2
        self.height = math.floor( (self.height - 1) / 2 + 1)
        print("Height is now:", self.height, "Width is now:", self.width)
        
        
        in_channels = 64
        
        num_repeat = self.layer_dict[n_layers]
        for i in range(2,6):
            self.res_layer = i
            # [ [blocks], transition ]
            self.layers[self.res_layer] = [[], None]
            for j in range(num_repeat[i-2]):
                self.create_block(in_channels, out_channels, j, bottleneck)
                if j == 0:
                    self.add_transition()
                in_channels = out_channels
            out_channels = out_channels * 2
            
        # global average pooling
        self.global_avg = nn.AvgPool2d(kernel_size = (self.width,self.height), stride = 1)
        # fully connected to final
        self.output = nn.Linear(in_channels,1)
        
        
        
        
    def create_block(self, in_channels, out_channels, block_num, bottleneck):
        block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, stride = 1, **self.conv_params),
            nn.BatchNorm2d(num_features = out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, stride = 2 if block_num == 0 else 1, **self.conv_params),
            nn.BatchNorm2d(num_features = out_channels),
            nn.ReLU()
        )
        self.add_module("conv" + str(self.res_layer) + "_" + str(block_num), block)
        self.layers[self.res_layer][0].append(block)
        print("Added", "conv" + str(self.res_layer) + "_" + str(block_num), "input: " + str(in_channels), "output: " + str(out_channels))
        
        if block_num == 0:
            self.height = math.floor( (self.height - 1) / 2 + 1)
            self.width = math.floor( (self.width - 1) / 2 + 1)
            print("Height is now:", self.height, "Width is now:", self.width)
        
    
    def add_transition(self):
        transition = nn.Sequential(
            nn.AvgPool2d(kernel_size = 3, stride = 2, padding = 1)
        )
        self.add_module("transition"+ str(self.res_layer), transition)
        self.layers[self.res_layer][1] = transition
    
    def forward(self, X):
        # go through conv1
        X = self.conv1(X)
        # go through residuals
        for i in range(2,self.res_layer + 1):
            layers,transition = self.layers[i]
            for j,layer in enumerate(layers):
                if j == 0:
                    
                    pool = transition(X)
                    # dimension transition
                    if i > 2:
                        padding = (0,0,0,0,pool.shape[1]//2,pool.shape[1]//2,0,0)
                        pool = nn.functional.pad(pool,padding)
                    X = layer(X)
                    X = X + pool
                else:
                    X = layer(X)
        X = self.global_avg(X)
        X = X.view(X.shape[0],-1)
        X = self.output(X)
        return X.view(-1)
        

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
resnet34 = ResNet(34,1).to(device)

Height is now: 43 Width is now: 47
Added conv2_0 input: 64 output: 64
Height is now: 22 Width is now: 24
Added conv2_1 input: 64 output: 64
Added conv2_2 input: 64 output: 64
Added conv3_0 input: 64 output: 128
Height is now: 11 Width is now: 12
Added conv3_1 input: 128 output: 128
Added conv3_2 input: 128 output: 128
Added conv3_3 input: 128 output: 128
Added conv4_0 input: 128 output: 256
Height is now: 6 Width is now: 6
Added conv4_1 input: 256 output: 256
Added conv4_2 input: 256 output: 256
Added conv4_3 input: 256 output: 256
Added conv4_4 input: 256 output: 256
Added conv4_5 input: 256 output: 256
Added conv5_0 input: 256 output: 512
Height is now: 3 Width is now: 3
Added conv5_1 input: 512 output: 512
Added conv5_2 input: 512 output: 512


In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [9]:
count_parameters(resnet18)

21112705

### Training

In [10]:
optim = torch.optim.Adam(resnet18.parameters(),lr = 0.01, betas = (0.9,0.999))
loss_metric = nn.L1Loss()
n_epochs = 10
iteration = 0
for e in range(n_epochs):
    losses = []
    for batch_input, batch_labels in dataloader:
        if iteration % 25 == 0:
            print(iteration)
        # make sure to zero out gradient
        resnet18.zero_grad()
        
        # move to gpu + get correct labels
        batch_input = batch_input.to(device)
        batch_labels = label_mapping_scaled[batch_labels].to(device)
        
        loss = loss_metric(resnet18(batch_input),batch_labels)
        losses.append(loss.data)
        loss.backward()
        optim.step()
        iteration += 1
#         break
    print("Epoch %d: Training Loss: %0.3f" % (e,np.mean(losses)))

0
25
50
75
100
125
150
175
Epoch 0: Training Loss: 0.164
200
225
250
275
300
325
350
Epoch 1: Training Loss: 0.104
375
400
425
450
475
500
525
Epoch 2: Training Loss: 0.090
550
575
600
625
650
675
700
Epoch 3: Training Loss: 0.084
725
750
775
800
825
850
875
Epoch 4: Training Loss: 0.078
900
925
950
975
1000
1025
1050
Epoch 5: Training Loss: 0.071
1075
1100
1125
1150
1175
1200
1225
1250
Epoch 6: Training Loss: 0.071
1275
1300
1325
1350
1375
1400
1425
Epoch 7: Training Loss: 0.067
1450
1475
1500
1525
1550
1575
1600
Epoch 8: Training Loss: 0.066
1625
1650
1675
1700
1725
1750
1775
Epoch 9: Training Loss: 0.061
